In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5io
import pandas as pd
from meeglet import define_frequencies

from scipy import io
import scipy.misc 
import statsmodels.api as sm

import seaborn as sns
import pyriemann
from pyriemann.stats import PermutationDistance
from coffeine.spatial_filters import ProjCommonSpace
import seaborn as sns
from scipy import stats

In [ ]:
print(scipy.__version__)


In [ ]:
foi = define_frequencies(foi_start=1, foi_end=64, bw_oct=0.35, delta_oct=0.05)[0]

### GROUP LEVEL STATISTICS

### all features

In [ ]:
features_CBU = h5io.read_hdf5('./meeglet_CBU_2023-06-22_10-06.h5')
features_CTB1 = h5io.read_hdf5('./meeglet_CTB_2023-06-22_10-46.h5')
features_CTB2 = h5io.read_hdf5('./meeglet_CTB_2023-06-23_11-04.h5')

In [ ]:
# merge 3 hdf5 files (key: patient, value : tuple cov, pow)
features_all = features_CTB1 | features_CTB2 | features_CBU

# Build subjects group and import features

In [ ]:
participants_fname = './participants.tsv'
subject_df = pd.read_csv(participants_fname, delimiter='\t')
subject_df['participant_id'] = subject_df['participant_id'].str.replace('sub-', '')
subject_df = subject_df.set_index('participant_id')

In [ ]:
subject_df = subject_df.loc[features_all.keys()] # dataframe in correct order (without the patient with no eeg)

In [ ]:
feature_converter = subject_df['Converters'].fillna(-1)

In [ ]:
# import rplain
r_plain = np.array([features_all[subject][6] for subject in features_all]) # 0 cov, 1 pow, 2 csd
r_plain.shape

In [ ]:
del features_all, features_CBU, features_CTB1, features_CTB2

## regularize r_plain

In [ ]:
n_compo = 65
r_plain_reg = np.empty_like(r_plain[:,:n_compo,:n_compo]) # init output array
for i in range(r_plain.shape[3]): #loops on frequencies
    regularizer = ProjCommonSpace(n_compo = n_compo, reg=1e-15, scale=1)
    r_plain_sym = pyriemann.utils.base.nearest_sym_pos_def(r_plain[:,:,:,i])
    r_plain_reg[:, :, :, i] = np.array(
        list(regularizer.fit_transform(r_plain_sym)['cov'])
    ) # method fit applied on the covs for subject i

## pyriemann MANOVA on r_plain

# converters

In [ ]:
from pyriemann.stats import PermutationDistance

labels = np.array(feature_converter)

pv = []
Fv = []
for i in range(r_plain_reg.shape[3]):
    p_test = PermutationDistance(1000, metric='riemann', mode='pairwise') 
    p, F = p_test.test(r_plain_reg[:,:,:,i], labels, verbose = False)
    pv.append(p)
    Fv.append(F[0])

In [ ]:
# Convert lists to numpy arrays for easier saving
pv = np.array(pv)
Fv = np.array(Fv)

# Save p-values and F-scores as .npy files
np.save('./p_values_rplain.npy', pv)
np.save('./F_scores_rplain.npy', Fv)

In [ ]:
# Load the previously saved results
pv = np.load('./p_values_rplain.npy')
Fv = np.load('./F_scores_rplain.npy')

## Script to plot Figures with FDR pvalues

In [ ]:
import seaborn as sns
plt.rcParams['font.size'] = 11
fig, axes = plt.subplots(1, 1, figsize=[3.8,2.1], sharey=True)
sig = 0.05
axes.plot(foi, Fv, lw=1.3, c='k')
plt.xlabel('Frequencies (Hz)')
plt.ylabel('MANOVA F Score')

# Fill the area between MANOVA F scores and threshold line with green
sig_index = np.array(pv) < sig
axes.fill_between(foi, Fv, 1, where=sig_index, interpolate=True, color='g', alpha=0.3)

# Customize legend
sig = pv[pv < 0.05].max()
legend = axes.legend(['F Score', 'p < %.2f' % sig])
axes.set_title('Power envelope MANOVA', fontsize = 11, y=0.95)
legend.set_frame_on(False)

# Set x-axis to display full numbers
axes.set_xscale('log', base=2)
xticks = [1, 2, 4, 8, 16, 32, 64]
yticks = [1, 2, 3]
axes.set_xticks(xticks)
axes.set_xticklabels(xticks)
axes.set_yticks(yticks)
axes.set_yticklabels(yticks)
#axes.set_ylim(1,7)
axes.set_xlim(2,64)
sns.despine(offset=4, trim=False);

axes2 = axes.twinx()
axes2.plot(foi, -np.log10(stats.false_discovery_control(pv)), color='orange')
axes2.set_ylabel('p value' + ' (FDR)', color='orange')
axes2.set_ylim(1, 4)
axes2.axhline(-np.log10(0.05), color='red', linestyle='--')
axes2.annotate(xy=(0.7, 0.14), text='p = 0.05', color='r', xycoords='axes fraction')
axes2.set_yticks(-np.log10([0.1, 0.01, 0.001, 0.0001]))
axes2.set_yticklabels([0.1, 0.01, 0.001, 0.0001], color='orange')
# Remove top and right edges
sns.despine(right=True, top=True, ax=axes)
sns.despine(right=False, left=True, top=True, bottom=True, ax=axes2)
plt.savefig('./figures/figure_rplain_manova_65compo_FDR_Denis.png', dpi=300, bbox_inches='tight')